<a href="https://colab.research.google.com/github/tvani2/Neural-Networks-Facial-Expression-Recognition-Challenge/blob/main/Expr_recognition_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle wandb onnx -Uq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 46.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/cs231n/assignments/assignment4/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
! unzip challenges-in-representation-learning-facial-expression-recognition-challenge

 88% 250M/285M [00:00<00:00, 637MB/s] 
100% 285M/285M [00:02<00:00, 121MB/s]
Archive:  challenges-in-representation-learning-facial-expression-recognition-challenge.zip
  inflating: example_submission.csv  
  inflating: fer2013.tar.gz          
  inflating: icml_face_data.csv      
  inflating: test.csv                
  inflating: train.csv               


In [4]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tvani22 (tvani22-free-university-of-tbilisi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
# Change the file path to the correct location after unzipping
df = pd.read_csv('./train.csv')
X = df['pixels']
y = df['emotion']

train_size = 0.70
val_size = 0.15
test_size = 0.15
X_temp, X_test_new, y_temp, y_test_new = train_test_split(
    X, y, test_size=test_size, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=(val_size / (train_size + val_size)), random_state=42, stratify=y_temp
)

In [6]:
from torch.utils.data import Dataset

def fast_process_pixels(pixel_series):
    pixel_lists = pixel_series.str.split()
    pixel_array = np.array(pixel_lists.tolist(), dtype=np.float32)
    return pixel_array.reshape(-1, 48, 48, 1) / 255.0

X_train_normalized = fast_process_pixels(X_train)
X_val_normalized = fast_process_pixels(X_val)
X_test_new_normalized = fast_process_pixels(X_test_new)

print("Data preprocessing completed!")
print(f"Train shape: {X_train_normalized.shape}")
print(f"Val shape: {X_val_normalized.shape}")
print(f"Test shape: {X_test_new_normalized.shape}")

# === 4. Dataset Class ===
class FastEmotionDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = torch.from_numpy(images).permute(0, 3, 1, 2).float()
        self.labels = torch.from_numpy(labels.values).long()
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

Data preprocessing completed!
Train shape: (20095, 48, 48, 1)
Val shape: (4307, 48, 48, 1)
Test shape: (4307, 48, 48, 1)


In [7]:
import torch
from torchvision import transforms
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import WeightedRandomSampler, DataLoader

# === 5. Transforms ===
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.Normalize([0.5], [0.5])
])

val_test_transforms = transforms.Compose([
    transforms.Normalize([0.5], [0.5])
])

# === 6. Create Datasets ===
train_dataset = FastEmotionDataset(X_train_normalized, y_train, transform=train_transforms)
val_dataset = FastEmotionDataset(X_val_normalized, y_val, transform=val_test_transforms)
test_dataset = FastEmotionDataset(X_test_new_normalized, y_test_new, transform=val_test_transforms)

# === 7. Compute Class Weights and Sampler ===
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(zip(np.unique(y_train), class_weights))
sample_weights = np.array([class_weights_dict[label] for label in y_train])
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# === 8. DataLoaders ===
batch_size = 64
num_workers = 0
pin_memory = torch.cuda.is_available()

train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler,
                          num_workers=num_workers, pin_memory=pin_memory,
                          persistent_workers=num_workers > 0)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=num_workers, pin_memory=pin_memory,
                        persistent_workers=num_workers > 0)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=pin_memory,
                         persistent_workers=num_workers > 0)

In [8]:
class EmotionDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        # Convert to torch tensors and rearrange dimensions (H, W, C) -> (C, H, W)
        self.images = torch.FloatTensor(images).permute(0, 3, 1, 2)
        self.labels = torch.LongTensor(labels.values)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define data augmentation transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
])

# No augmentation for validation and test
val_test_transforms = transforms.Compose([
    transforms.Normalize(mean=[0.5], std=[0.5])  # Same normalization as training
])

# Create datasets
train_dataset = EmotionDataset(X_train_normalized, y_train, transform=train_transforms)
val_dataset = EmotionDataset(X_val_normalized, y_val, transform=val_test_transforms)
test_dataset = EmotionDataset(X_test_new_normalized, y_test_new, transform=val_test_transforms)

In [9]:
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Create sample weights for each training sample
sample_weights = [class_weights_dict[label] for label in y_train]
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

In [10]:
# Create DataLoaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_new_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print class distribution info
print("Class distribution in training set:")
print(y_train.value_counts().sort_index())
print(f"\nClass weights: {class_weights_dict}")

Class distribution in training set:
emotion
0    2797
1     306
2    2867
3    5051
4    3380
5    2219
6    3475
Name: count, dtype: int64

Class weights: {0: np.float64(1.0263547678635272), 1: np.float64(9.381419234360411), 2: np.float64(1.0012955304200508), 3: np.float64(0.568345730689821), 4: np.float64(0.849323753169907), 5: np.float64(1.2936972896414085), 6: np.float64(0.8261048304213772)}


In [11]:
import torch.nn as nn
import torchvision.models as models

class ImprovedEmotionResNet(nn.Module):
    def __init__(self, num_classes=7, dropout_rate=0.5):
        super(ImprovedEmotionResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=True)

        # Replace the first layer for grayscale input
        self.resnet.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self._add_dropout_layers()

        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_features, num_features // 2),
            nn.BatchNorm1d(num_features // 2),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate * 0.7),
            nn.Linear(num_features // 2, num_classes)
        )

    def _add_dropout_layers(self):
        self.resnet.layer1 = nn.Sequential(self.resnet.layer1, nn.Dropout2d(0.1))
        self.resnet.layer2 = nn.Sequential(self.resnet.layer2, nn.Dropout2d(0.15))
        self.resnet.layer3 = nn.Sequential(self.resnet.layer3, nn.Dropout2d(0.2))
        self.resnet.layer4 = nn.Sequential(self.resnet.layer4, nn.Dropout2d(0.25))

    def forward(self, x):
        return self.resnet(x)

In [12]:
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')

    def __call__(self, val_loss, model=None):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import wandb
from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImprovedEmotionResNet(num_classes=7, dropout_rate=0.5).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0002, weight_decay=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)
early_stopping = EarlyStopping(patience=10, min_delta=0.001)

wandb.init(project="Facial_Expression_Recognition_1")
wandb.watch(model, log="all", log_freq=100)

model_save_path = "best_resnet_model.pth"
best_val_loss = float('inf')

for epoch in range(20):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_train_loss = running_loss / len(train_loader.dataset)
    train_acc = 100. * correct / total

    # Validation
    model.eval()
    val_loss_total, correct_val, total_val = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss_total += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()

    avg_val_loss = val_loss_total / len(val_loader.dataset)
    val_acc = 100. * correct_val / total_val

    print(f"Epoch {epoch+1:02d}: Train Loss = {avg_train_loss:.4f}, Train Acc = {train_acc:.2f}%, "
          f"Val Loss = {avg_val_loss:.4f}, Val Acc = {val_acc:.2f}%")

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "train_accuracy": train_acc,
        "val_loss": avg_val_loss,
        "val_accuracy": val_acc,
        "learning_rate": optimizer.param_groups[0]['lr']
    })

    scheduler.step(avg_val_loss)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), model_save_path)

    if early_stopping(avg_val_loss):
        print(f"Early stopping at epoch {epoch+1}")
        break

# Load best model
model.load_state_dict(torch.load(model_save_path))
model.eval()

# Final Test Evaluation
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy_score(all_labels, all_preds) * 100
print(f"\n✅ Test Accuracy: {test_accuracy:.2f}%")
wandb.finish()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 210MB/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

Epoch 01: Train Loss = 1.9601, Train Acc = 18.15%, Val Loss = 1.8352, Val Acc = 24.66%
Epoch 02: Train Loss = 1.7362, Train Acc = 32.21%, Val Loss = 1.6079, Val Acc = 36.89%
Epoch 03: Train Loss = 1.5743, Train Acc = 39.73%, Val Loss = 1.4937, Val Acc = 42.54%
Epoch 04: Train Loss = 1.4833, Train Acc = 43.81%, Val Loss = 1.3840, Val Acc = 47.20%
Epoch 05: Train Loss = 1.4071, Train Acc = 47.23%, Val Loss = 1.3788, Val Acc = 46.92%
Epoch 06: Train Loss = 1.3622, Train Acc = 49.26%, Val Loss = 1.3818, Val Acc = 47.85%
Epoch 07: Train Loss = 1.3074, Train Acc = 51.11%, Val Loss = 1.3060, Val Acc = 50.62%
Epoch 08: Train Loss = 1.2789, Train Acc = 52.38%, Val Loss = 1.4617, Val Acc = 46.64%
Epoch 09: Train Loss = 1.2498, Train Acc = 53.70%, Val Loss = 1.2464, Val Acc = 53.01%
Epoch 10: Train Loss = 1.2220, Train Acc = 54.93%, Val Loss = 1.2520, Val Acc = 52.96%
Epoch 11: Train Loss = 1.1997, Train Acc = 55.45%, Val Loss = 1.2364, Val Acc = 54.42%
Epoch 12: Train Loss = 1.1751, Train Acc = 

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▃▅▅▆▆▆▇▇▇▇▇████████
train_loss,█▆▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
val_accuracy,▁▄▅▆▆▆▇▆▇▇▇▇██▇█████
val_loss,█▆▄▃▃▃▃▄▂▂▂▂▁▂▂▁▁▁▁▁
epoch,20
learning_rate,0.0002
train_accuracy,59.90047
train_loss,1.0858
val_accuracy,56.97701
